In [6]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.6 MB/s eta 0:00:00


In [1]:
from dataclasses import dataclass
import math

import torch
import torch.nn as nn
from torch.nn import functional as F
from transformers import GPT2LMHeadModel


@dataclass
class GPTConfig:
    """This class defines the configuration for the GPT model."""

    block_size: int = 1024
    vocab_size: int = 50257

    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768


class CausalSelfAttention(nn.Module):
    """Attention module."""

    def __init__(self, config: GPTConfig) -> None:
        """Initialize MLP."""
        super().__init__()
        # Batch of key/query/value projects for all heads
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # Output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        # Regularization
        self.n_head = config.n_head
        self.n_embed = config.n_embd
        self.register_buffer(
            "bias",
            torch.tril(torch.ones(config.block_size, config.block_size)).view(
                1, 1, config.block_size, config.block_size
            ),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Perform inference."""
        B, T, C = x.size()
        # Compute the query, key, value for all heads in the batch.
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embed, dim=2)
        # Each are (B, nh, T, hs)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        # attention materializes (T, T)
        # Queries and keys interact
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        # Ensure tokens only attend to tokens before them and not to tokens in the future
        att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float("-inf"))
        # Normalize attention
        att = F.softmax(att, dim=-1)
        # Compute a weighted sum of interesting tokens
        y = att @ v
        # Reassemble and concat everything
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        # Output projection
        y = self.c_proj(y)
        return y


class MLP(nn.Module):
    """Multi-layer perceptron."""

    def __init__(self, config: GPTConfig) -> None:
        """Initialize MLP."""
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu = nn.GELU(approximate="tanh")
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Perform inference."""
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x


class Block(nn.Module):
    """A transformer block."""

    def __init__(self, config: GPTConfig) -> None:
        """Initialize Block."""
        super().__init__()
        self.config = config
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Perform inference."""
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

In [2]:

class GPT(nn.Module):
    """This class defines the GPT model."""

    def __init__(self, config: GPTConfig) -> None:
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(
            {
                "wte": nn.Embedding(config.vocab_size, config.n_embd),
                "wpe": nn.Embedding(config.block_size, config.n_embd),
                "h": nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
                "ln_f": nn.LayerNorm(config.n_embd),
            }
        )
        # Final classifier
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Perform generation."""
        B, T = x.size()
        assert T <= self.config.block_size  # Max sequence length
        # Forward token and positional embeddings
        pos = torch.arange(0, T, dtype=torch.long, device=x.device)  # Shape (T)
        pos_emb = self.transformer.wpe(pos)  # (T, n_emb)
        tok_emb = self.transformer.wte(x)  # (B, T, n_emb)
        x = tok_emb + pos_emb
        # Forward transformer blocks
        for block in self.transformer.h:
            x = block(x)
        # Forward the final layernorm
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)  # (B, T, vocab_size)
        return logits

    @classmethod
    def from_pretrained(cls, model_type: str) -> "GPT":
        """Load the GPT from the pretrained model."""
        assert model_type in {"gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl"}
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            "gpt2": dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            "gpt2-medium": dict(n_layer=24, n_head=16, n_embd=1024),  # 350M params
            "gpt2-large": dict(n_layer=36, n_head=20, n_embd=1280),  # 774M params
            "gpt2-xl": dict(n_layer=48, n_head=25, n_embd=1600),  # 1558M params
        }[model_type]
        config_args["vocab_size"] = 50257  # always 50257 for GPT model checkpoints
        config_args["block_size"] = 1024  # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [
            k for k in sd_keys if not k.endswith(".attn.bias")
        ]  # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [
            k for k in sd_keys_hf if not k.endswith(".attn.masked_bias")
        ]  # ignore these, just a buffer
        sd_keys_hf = [
            k for k in sd_keys_hf if not k.endswith(".attn.bias")
        ]  # same, just the mask (buffer)
        transposed = [
            "attn.c_attn.weight",
            "attn.c_proj.weight",
            "mlp.c_fc.weight",
            "mlp.c_proj.weight",
        ]

        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(
            sd_keys
        ), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

In [4]:
num_return_sequences = 5
max_length = 30

device = torch.device("cuda")

model = GPT.from_pretrained("gpt2")
model.eval()
model.to(device)

loading weights from pretrained gpt: gpt2


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [7]:
import tiktoken

enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode("Hello, I'm a language model,")
tokens = torch.tensor(tokens, dtype=torch.long) # (8, )
# Replicate input tokens
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)

# x is (B, T)
x = tokens.to(device)

In [10]:
# With each loop iteration we'll append a token to the sequence. This is
# adding one more column to x each time.
torch.manual_seed(42)
torch.cuda.manual_seed(42)
while x.size(1) < max_length:
  with torch.no_grad():
    print(f"Generating ({x.shape})")
    logits = model(x)  # (B, T, vocab_size)
    # Take the logits at the last position (next character) and drop the others.
    # This is correct but inefficient implementation of sampling.
    # Question: What is T?
    logits = logits[:, -1, :]  # (B, vocab_size)
    probs = F.softmax(logits, dim=-1)
    # Do top-k sampling of 50 which is the huggingface default. Get the top 50
    # probabilities and set all other tokens to probability of zero. This helps
    # keep the model on track so it doesn't go off the rails as easily.
    # Both are (5, 50)
    topk_probs, topk_indicies = torch.topk(probs, 50, dim=-1)
    # Select a token from the top 5
    ix = torch.multinomial(topk_probs, 1)  # (B, 1)
    # Gather corresponding indicidies
    xcol = torch.gather(topk_indicies, -1, ix)
    # Append the new character to the sequence (one for each in the batch)
    x = torch.cat((x, xcol), dim=-1)

Generating (torch.Size([5, 8]))
Generating (torch.Size([5, 9]))
Generating (torch.Size([5, 10]))
Generating (torch.Size([5, 11]))
Generating (torch.Size([5, 12]))
Generating (torch.Size([5, 13]))
Generating (torch.Size([5, 14]))
Generating (torch.Size([5, 15]))
Generating (torch.Size([5, 16]))
Generating (torch.Size([5, 17]))
Generating (torch.Size([5, 18]))
Generating (torch.Size([5, 19]))
Generating (torch.Size([5, 20]))
Generating (torch.Size([5, 21]))
Generating (torch.Size([5, 22]))
Generating (torch.Size([5, 23]))
Generating (torch.Size([5, 24]))
Generating (torch.Size([5, 25]))
Generating (torch.Size([5, 26]))
Generating (torch.Size([5, 27]))
Generating (torch.Size([5, 28]))
Generating (torch.Size([5, 29]))


In [15]:

for i in range(num_return_sequences):
  tokens = x[i, :max_length].tolist()
  decoded = enc.decode(tokens)
  print("> ", decoded)



>  Hello, I'm a language model, not a program.

So this morning I started studying for the interview in the lab. This was not
>  Hello, I'm a language model, and one of the main things that bothers me when they create languages is how easy it becomes to create something that
>  Hello, I'm a language model, and I wrote it off on the grounds that a language model would make me more fluent. But I'm not
>  Hello, I'm a language model, I really like languages. I like languages because like, they're good. And the way we talk about languages
>  Hello, I'm a language model, a language model I'm using for data modelling. All I did was test the results and then I wrote some
